In [ ]:
__version__ = '1.0.0-rc.2'
__author__ = 'Martino Pulici'

In [ ]:
import csv

In [ ]:
import pandas as pd
import pulp

In [ ]:
from meal_plan_optimizer import *

In [ ]:
nutrient_limits = {'Energy': 2000,
                   'Fat': 70,
                   'Saturates': 20,
                   'Carbohydrates': 260,
                   'Sugars': 90,
                   'Protein': 50
                   }
# nutrient daily limits

In [ ]:
df = pd.read_csv("data/food.csv")
# ingredient nutrients file

with open("data/dish.csv") as f:
    # dishes file
    reader = csv.reader(f)
    dishes_list = list(reader)
    # dishes strings list

with open("data/unlimited.txt") as f:
    # unlimited dishes file
    reader = csv.reader(f)
    unlimited_dishes = list(reader)[0]
    # unlimited dishes list

In [ ]:
labels = list(df['Food'])
# list of food names
costs = dict(zip(labels, df['Cost']))
# dictionary of food costs

In [ ]:
nutrients = {}
# dictionary of food nutrients

for i in range(len(labels)):
    # cycle on the foods
    nutrients[labels[i]] = {}
    # dictionary of nutrients for specific food

    for nut in nutrient_limits.keys():
        # cycle on nutrient limits
        nutrients[labels[i]][nut] = df[nut][i] * 10
        # nutrient quantities multiplied times 10 to convert to kg

In [ ]:
dishes_labels = []
# dish names list
dishes = []
# dishes list

for i in range(len(dishes_list)):
    # cycle on dishes strings list
    d = dishes_list[i]
    # current dish
    dishes.append(dish(d[0], nutrient_limits))
    # dish added to dishes list
    dishes_labels.append(d[0])
    # dish name appended to list

    for j in range(1, len(d), 2):
        # cycle on dish ingredients
        ingredient = d[j]
        # current ingredient
        quantity = float(d[j + 1]) / 1000
        # current quantity devided by 1000 to convert to kg
        dishes[i].cost += costs[ingredient] * quantity
        # current ingredient cost added to dish cost

        for nut in dishes[i].nutrients.keys():
            # cycle on ingredient nutrients
            dishes[i].nutrients[nut] += nutrients[ingredient][nut] * quantity
            # ingredient nutrient added to dish nutrient

In [ ]:
food_day_1 = pulp.LpVariable.dicts("Food_Day_1", dishes_labels, 0)
food_day_2 = pulp.LpVariable.dicts("Food_Day_2", dishes_labels, 0)
food_day_3 = pulp.LpVariable.dicts("Food_Day_3", dishes_labels, 0)
food_day_4 = pulp.LpVariable.dicts("Food_Day_4", dishes_labels, 0)
food_day_5 = pulp.LpVariable.dicts("Food_Day_5", dishes_labels, 0)

foods = [food_day_1, food_day_2, food_day_3, food_day_4, food_day_5]
# LpVariable list

In [ ]:
prob_day_1 = pulp.LpProblem("Day_1", pulp.LpMinimize)
prob_day_2 = pulp.LpProblem("Day_2", pulp.LpMinimize)
prob_day_3 = pulp.LpProblem("Day_3", pulp.LpMinimize)
prob_day_4 = pulp.LpProblem("Day_4", pulp.LpMinimize)
prob_day_5 = pulp.LpProblem("Day_5", pulp.LpMinimize)

probs = [prob_day_1, prob_day_2, prob_day_3, prob_day_4, prob_day_5]
# LpProblems list

In [ ]:
for i in range(len(probs)):
    # cycle on problems
    probs[i] += pulp.lpSum([dish.cost * foods[i][dish.name]
                           for dish in dishes])
    # objective added to problem

    for nut in nutrient_limits.keys():
        # cycle on nutrients
        probs[i] += pulp.lpSum([dish.nutrients[nut] * foods[i][dish.name]
                               for dish in dishes]) >= nutrient_limits[nut] * 0.67
        # minimum nutrient quantity constraint added to problem
        probs[i] += pulp.lpSum([dish.nutrients[nut] * foods[i][dish.name]
                               for dish in dishes]) <= nutrient_limits[nut] * 1.5
        # maximum nutrient quantity constraint added to problem

    for j in range(len(probs[i].variables())):
        # cycle on dishes
        probs[i] += probs[i].variables()[j] <= 3
        # maximum number of servings for dish added

In [ ]:
int_probs = []
# integer problems list

for i in range(0, len(probs)):
    # cycle on problems

    for j in range(0, i):
        # cycle on previous problems

        for k in range(len(int_probs[j].variables())):
            # cycle on previous problem dishes

            if int_probs[j].variables()[k].varValue and not unlimited_dish(
                    str(int_probs[j].variables()[k]), unlimited_dishes):
                # entered if dish has already been chosen for a previous
                # problem and it is not an unlimited dish
                probs[i] += probs[i].variables()[k] == 0
                # dish quantity set to 0

            if (j == i - 1 or j == i - 2) and int_probs[j].variables()[
                    k].varValue and unlimited_dish(str(int_probs[j].variables()[k]), unlimited_dishes):
                # entered if dish is already chosen for one of the last two
                # problems and it is an unlimited dish
                probs[i] += probs[i].variables()[k] == 0
                # dish quantity set to 0

    int_probs.append(branch_and_bound(probs[i]))
    # integer problem added to list
    int_probs[-1].solve()
    # last problem solved

In [ ]:
print("Total cost = " +
      "{0:.2f}".format(pulp.value(sum([n.objective for n in int_probs]))) +
      " €")

for day in ["_Day_1_", "_Day_2_", "_Day_3_", "_Day_4_", "_Day_5_"]:
    print()
    print(day[1:6].upper())

    for prob in int_probs:
        for v in prob.variables():
            if v.varValue and day in v.name:
                print(v.name[11:] + " = {0:.0f}".format(v.varValue))